In [19]:
import os, shutil
from glob import glob
import json

import numpy as np

In [45]:
root_p = './data/ori_label/file_1/'
path = './data/ori_label/file_1/file_1.json'

js = json.load(open(path))

name = path.split('/')[-1].replace('.json','')
new_dir_path = root_p.replace(name,'new_'+name)

os.makedirs(new_dir_path,exist_ok=True)

pos = js.get('sequence')['2d_pos']
label = js.get('action')

with open(new_dir_path+'/'+name+'.json', 'w') as j:
    json.dump({'label':label, 'pos':pos}, j)


In [95]:
root_p = './data/ori_label'
folder_list = glob(root_p+'/*')
for folder in folder_list : 
    
    json_list = glob(folder + '/*.json')
    for json_file in json_list:

        js = json.load(open(json_file))
        name = json_file.split('\\')[-1].replace('.json','')
        new_dir_path = folder.replace('ori_label','new_label')
        os.makedirs(new_dir_path,exist_ok=True)
        
        pos = js.get('sequence')['2d_pos']
        label = js.get('action')
        
        with open(new_dir_path+'/'+name+'.json', 'w') as j:
            json.dump({'label':label, 'pos':pos}, j)
    

(141, 72)
(141, 72)
(141, 72)


# read 2d-pos

In [59]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset

import numpy as np
from sklearn.preprocessing import MinMaxScaler

import os, shutil
from glob import glob
import json

import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


root_p = './data/ori_label'
folder_list = glob(root_p+'/*')
pos_set = []
label_set = []
for folder in folder_list : 
    
    json_file = glob(folder + '/*.json')[0]

    js = json.load(open(json_file))
    name = json_file.split('\\')[-1].replace('.json','')
    new_dir_path = folder.replace('ori_label','new_label')

    pos = np.array(js.get('sequence')['2d_pos'], dtype=np.float64).tolist()
    pos_set.extend(pos)
    label = [int(js.get('action'))] * len(pos)
    label_set.extend(label)
print(np.array(pos_set).shape)
print(np.array(label_set).shape)

(22218, 72)
(22218,)


In [60]:
# split = 20000


# train_pos = torch.FloatTensor(pos_set[:split]).to(device)
# train_label = torch.FloatTensor(label_set[:split]).to(device)

# valid_pos = torch.FloatTensor(pos_set[split:]).to(device)
# valid_label = torch.FloatTensor(label_set[split:]).to(device)

# print(np.array(train_pos).shape, np.array(train_label).shape)
# print(np.array(valid_pos).shape, np.array(valid_label).shape)

In [68]:
def seq_data(x, y, sequence_length):
  
    x_seq = []
    y_seq = []
    for i in range(len(x) - sequence_length):
        x_seq.append(x[i: i+sequence_length])
        y_seq.append(y[i+sequence_length])

    return torch.FloatTensor(x_seq).to(device), torch.FloatTensor(y_seq).to(device).view([-1, 1])

In [69]:
pos_seq, label_seq = seq_data(pos_set, label_set, sequence_length=70)

print(pos_seq.shape)
print(label_seq.shape)

split = 20000

train_pos = pos_seq[:split]
train_label = label_seq[:split]

valid_pos = pos_seq[split:]
valid_label = label_seq[split:]

torch.Size([22148, 70, 72])
torch.Size([22148, 1])


In [70]:
train = torch.utils.data.TensorDataset(train_pos, train_label)
valid = torch.utils.data.TensorDataset(valid_pos, valid_label)

train_set = DataLoader(dataset=train, batch_size=20, shuffle=False)
valid_set = DataLoader(dataset=valid, batch_size=20, shuffle=False)

In [71]:
input_size = 72
num_layers = 2
hidden_layer = 8
num_class = 6
sequence_length = 70

In [72]:
class VanillaRNN(nn.Module):
    def __init__(self, input_size, hidden_layer, sequence_length, num_class, num_layers, device):
        super(VanillaRNN, self).__init__()
        self.device = device
        self.hidden_layer = hidden_layer
        self.num_class = num_class
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_layer, num_layers, batch_first=True)
        self.fc = nn.Sequential(nn.Linear(hidden_layer * sequence_length, self.num_class), nn.Softmax())

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_layer).to(self.device) # 초기 hidden state 설정하기.
        out, _ = self.rnn(x, h0) # out: RNN의 마지막 레이어로부터 나온 output feature 를 반환한다. hn: hidden state를 반환한다.
        out = out.reshape(out.shape[0], -1) # many to many 전략
        out = self.fc(out)
        return out
    
model = VanillaRNN(input_size=input_size,
                   hidden_layer=hidden_layer,
                   sequence_length=sequence_length,
                   num_class=num_class,
                   num_layers=num_layers,
                   device=device).to(device)

In [73]:
criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 200
optimizer = optim.Adam(model.parameters(), lr=lr)

In [74]:
loss_graph = [] # 그래프 그릴 목적인 loss.
n = len(train_set)

for epoch in range(num_epochs):
    running_loss = 0.0

    for data in train_set:

        seq, target = data # 배치 데이터.
        out = model(seq)   # 모델에 넣고,
        loss = criterion(out, target) # output 가지고 loss 구하고,

        optimizer.zero_grad() # 
        loss.backward() # loss가 최소가 되게하는 
        optimizer.step() # 가중치 업데이트 해주고,
        running_loss += loss.item() # 한 배치의 loss 더해주고,

    loss_graph.append(running_loss / n) # 한 epoch에 모든 배치들에 대한 평균 loss 리스트에 담고,
#     if epoch % 100 == 0:
    print('[epoch: %d] loss: %.4f'%(epoch, running_loss/n))

[epoch: 0] loss: 8.1708
[epoch: 1] loss: 8.1708
[epoch: 2] loss: 8.1708
[epoch: 3] loss: 8.1707
[epoch: 4] loss: 8.1707
[epoch: 5] loss: 8.1707
[epoch: 6] loss: 8.1707
[epoch: 7] loss: 8.1707
[epoch: 8] loss: 8.1707
[epoch: 9] loss: 8.1707
[epoch: 10] loss: 8.1707
[epoch: 11] loss: 8.1707
[epoch: 12] loss: 8.1707
[epoch: 13] loss: 8.1707


KeyboardInterrupt: 

In [56]:
import torch
import torch.nn as nn


In [73]:
input_size = 150528
hidden_size = 8

# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 144, 150528)
print(inputs.shape)
cell = nn.RNN(input_size, hidden_size, batch_first=True)
outputs, _status = cell(inputs)

torch.Size([1, 144, 150528])


In [74]:
print(outputs.shape)
#모든 time-step의 hidden_state

torch.Size([1, 144, 8])


In [75]:
print(_status.shape)
# 마지막 시점에서의 상태

torch.Size([1, 1, 8])


In [81]:
inputs = torch.Tensor(1, 144, 150528)

cell = nn.RNN(input_size = 150528, hidden_size = 8, num_layers = 2, batch_first=True)

outputs, _status = cell(inputs)

In [82]:
print(outputs.shape)
print(_status.shape)

torch.Size([1, 144, 8])
torch.Size([2, 1, 8])


In [83]:
inputs = torch.Tensor(1, 144, 150528)

cell = nn.RNN(input_size = 150528, hidden_size = 8, 
              num_layers = 2, batch_first=True,
             bidirectional = True)

outputs, _status = cell(inputs)

In [84]:
print(outputs.shape)
print(_status.shape)

torch.Size([1, 144, 16])
torch.Size([4, 1, 8])


In [85]:
inputs = torch.Tensor(1, 144, 150528)

cell = nn.LSTM(input_size = 150528, hidden_size = 8, 
              num_layers = 2, batch_first=True,
             bidirectional = True)

outputs, _status = cell(inputs)

In [88]:
print(outputs.shape)
print(_status[0].shape)

torch.Size([1, 144, 16])
torch.Size([4, 1, 8])
